In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import financials


In [2]:
m = wt.watertap_setup(dynamic = False)
wt.case_study_trains.train = {"case_study": "Carlsbad",
                             "reference": "NAWI",
                             "scenario": "Baseline"}

wt.case_study_trains.source_water = {"case_study": "Carlsbad", 
                             "reference": "NAWI",
                             "scenario": "Baseline",
                             "water_type": "Seawater"}

In [3]:
import idaes.core.util.scaling as iscale
iscale.calculate_scaling_factors(m)

In [4]:
train_constituent_list = ["tds"]

In [5]:
m.fs.water = WaterParameterBlock()

In [6]:
financials.get_system_specs(m.fs, train=wt.case_study_trains.train)

Carlsbad


In [7]:
m = wt.design.add_water_source(m = m, source_name = "source1", reference = "NAWI", 
                               water_type = "Seawater", 
                     case_study = "Carlsbad", flow = 4.583) # m3/s 

adding source


In [8]:
import reverse_osmosis
setattr(m.fs, "reverse_osmosis", reverse_osmosis.UnitProcess(default={"property_package": m.fs.water}))
getattr(m.fs, "reverse_osmosis").get_costing(module=financials)

ValueError: Numeric value `50000*fs.reverse_osmosis.flow_vol_in[0.0]` (<class 'pyomo.core.expr.numeric_expr.MonomialTermExpression'>) is not in domain NonNegativeReals for Var fs.reverse_osmosis.membrane_area[0.0]

In [22]:
# m.fs.reverse_osmosis.flow_vol_in.fix(4.583)
# #m.fs.reverse_osmosis.flow_vol_in.unfix() #0.0001)
# m.fs.reverse_osmosis.conc_mass_in[:, "tds"].fix(35) 
# m.fs.reverse_osmosis.pressure_in.fix(1) 
# m.fs.reverse_osmosis.temperature_in.fix(300) 
#m.fs.reverse_osmosis.feed.pressure.fix()
m.fs.reverse_osmosis.membrane_area.unfix()
#m.fs.reverse_osmosis.feed.pressure.unfix()

In [89]:
#m = wt.design.add_unit_process(m = m, unit_process_name = "reverse_osmosis", unit_process_type = "reverse_osmosis")

In [18]:
#m.fs.reverse_osmosis.flow_vol_in.unfix()
#m.fs.reverse_osmosis.conc_mass_in[:, "tds"].unfix()
m.fs.arc1 = Arc(source=m.fs.source1.outlet, destination = getattr(m.fs, "reverse_osmosis").inlet)

In [19]:
#wt.display.show_train2(model_name=m)

In [20]:
#m.fs.reverse_osmosis.feed.pressure.unfix()

In [23]:
wt.run_water_tap(m = m, solver_results = True, print_model_results = True, objective=True)
print("In --->", m.fs.reverse_osmosis.feed.mass_flow_h20[0]())
print("Out --->", m.fs.reverse_osmosis.permeate.mass_flow_h20[0]())
print("Waste --->", m.fs.reverse_osmosis.retenate.mass_flow_h20[0]())

print("RO fixed_cap_inv cost", m.fs.reverse_osmosis.costing.fixed_cap_inv())
print("RO electricity_cost ", m.fs.reverse_osmosis.costing.electricity_cost())
#print("RO electricity_cost ", m.fs.reverse_osmosis.costing.total_up_cost())

degrees_of_freedom: 0
Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      100
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       28

Total number of variables............................:       40
                     variables with only lower bounds:       18
                variables with lower and upper bounds:        1
                     variables with only uppe

In [11]:
m.fs.costing.LCOW()

0.6384341762564585

In [12]:
m.fs.reverse_osmosis.membrane_area[0]()

888617.6842735186

In [13]:
50*1000*m.fs.reverse_osmosis.flow_vol_in[0]()

229150.0

In [38]:
ro_membrane_cost = 30 # $/m3
pump_cost = 53 #m3bar/h
saturated_saline_make_up = 0.025
membrane_replacement_factor = 15 #%of initial membrane area per year
maintenance_and_labor = 2 #% of initial investment/yr
chem_factor = 1 #% of init invest per year#/NA

In [41]:
membrane_area = 50

In [42]:
membrane_unit = ro_membrane_cost * membrane_area
pump_unit = pump_cost * m.fs.reverse_osmosis.pump_power()
pressure_exchanger = 0

In [51]:
# 1.4 in old RO. See Bartholomew et al. 2018
TCI = 1.4 * (membrane_unit + pump_unit) * 1e-6

In [53]:
tac = TCI*0.1 + 2*m.fs.reverse_osmosis.costing.electricity_cost()

In [54]:
tac*1e6 / (0.3*3600*24*365)

1.436024276021926

In [55]:
m.fs.costing.LCOW

In [17]:
feed=m.fs.reverse_osmosis.feed

In [18]:
value(8.45e7 * feed.osm_coeff[0]* feed.mass_frac_tds[0] / (1 - feed.mass_frac_tds[0]))

2783020.445065185

In [19]:
mass_frac_tds = 0.04952759752553286

In [20]:
value(8.45e7 * feed.osm_coeff[0]* mass_frac_tds / (1 - mass_frac_tds))

4081042.9868604005

In [24]:
1033.975186126418 * 0.04952759752553286 

51.21030686985716

In [12]:
value(m.fs.reverse_osmosis.mass_fraction_in[0])

0.010812025481409663

In [13]:
value(m.fs.reverse_osmosis.osmotic_coeff_in[0])

0.1

In [25]:
3974344.7699916563*1e-5

39.743447699916565

In [27]:
m.fs.reverse_osmosis.osmotic_coeff_in.display()

osmotic_coeff_in : water mass flow rate 
    Size=1, Index=fs.time
    Key : Lower : Value : Upper : Fixed : Stale : Domain
    0.0 :     0 :   0.1 :  None : False :  True : NonNegativeReals


In [20]:
value(m.fs.reverse_osmosis.mass_fraction_per[0])

2.175722122794811e-15

In [21]:
value(m.fs.reverse_osmosis.mass_fraction_ret[0])

4.668731502727216e-08

In [23]:
64.92996680681406 / (0.6312*64.92996680681406 + 997.86)

0.06250214624787276

In [ ]:
value(m.fs.reverse_osmosis.osmotic_pressure_in)

In [17]:
m.fs.reverse_osmosis.jw_per.display()

jw_per : water mass flow rate permeate
    Size=1, Index=fs.time
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
    0.0 :     0 : 0.9975000000022316 :  None : False : False : NonNegativeReals


In [18]:
m.fs.reverse_osmosis.js_per.display()

js_per : tds mass flow rate permeate
    Size=1, Index=fs.time
    Key : Lower : Value                 : Upper : Fixed : Stale : Domain
    0.0 :     0 : 3.142225908327049e-05 :  None : False : False : NonNegativeReals


In [ ]:
self.conc_mass_out_eq = Constraint(    
    expr=self.conc_mass_out[t, "tds"] == ((self.mass_flow_perm / self.flow_vol_out[t]) - 997.86) / 0.6312
) 

In [ ]:
3.142225908327049e-05 / 0.9975000000022316

In [124]:
a = 4.2E-7 # 𝑤𝑎𝑡𝑒𝑟 𝑝𝑒𝑟𝑚𝑒𝑎𝑏𝑖𝑙𝑖𝑡𝑦 coefficient m bar-1 s-1
p_in = 50 #bar pressure at inlet. should be unfixed.
p_atm = 1 #bar atmospheric pressure
pressure_drop = 3 # bar Typical pressure drops range from 0.1-3 bar.
p_ret = p_in - pressure_drop #bar atmospheric pressure
pw = 1000 # density of water kg/m3
membrane_area = 50 # per module m2

In [125]:
flow_in = 1
tds_in = 35.8
tds_per = tds_in * 0.002
tds_ret = tds_in - tds_per

In [126]:
def get_osmotic_pressure(tds):
    mass_flow_total = flow0.6312*tds + 997.86 #kg/s
    print(mass_flow_total)
    mass_fraction = tds / mass_flow_total #unitless
    osmotic_coeff = 4.92*mass_fraction**2 + mass_fraction*0.0889 + 0.918 
    osmotic_pressure = 8.45e7 * osmotic_coeff * mass_fraction / (1 - mass_fraction) #Pa
    osmotic_pressure = osmotic_pressure * 1e-5 #bar
    return osmotic_pressure


In [127]:
osmotic_pressure_in = get_osmotic_pressure(tds_in)
osmotic_pressure_out = get_osmotic_pressure(tds_ret)

1020.45696
1020.41176608


In [128]:
jw_out = ((p_out - p_atm) - osmotic_pressure_out)*pw*a #pout-pp goes to zero.
jw_in = ((p_in - p_atm) - osmotic_pressure_in)*pw*a #pout-pp goes to zero.
flow_per = membrane_area * 0.5 * (jw_in + jw_out) # permeate water flow kg/s
flow_per = flow_out #/ 997.86 # permeate water flow m3/s

print(flow_out2)

0.4136667895516468


In [107]:
b = 3.5E-8 # Salt permeability coefficient m s-1
salt_out = 0.5 * b * (tds_in + tds_out) * membrane_area # kg/m3

In [108]:
salt_out

1.14188375

In [ ]:
membrane_area = area_per_module * pure_water_perm / (pure_water_perm_manu)  # m^2


In [ ]:
x1 = 4.92*𝑋**2 + X*0.0889 + 0.918 
𝟖.𝟒𝟓𝑬𝟕 𝝓(𝑿)  𝑿/(𝟏−𝑿)  

In [ ]:
𝜌(𝑋)=756 𝑋+995 [𝑘𝑔/𝑚^3 ] 

In [ ]:
b = 3.5E-8 # Salt permeability coefficient m s-1


In [ ]:
Parameters
𝐴 −𝑤𝑎𝑡𝑒𝑟 𝑝𝑒𝑟𝑚𝑒𝑎𝑏𝑖𝑙𝑖𝑡𝑦 [𝑚·𝑠^(−1)·𝑃𝑎^(−1) ]
𝐵 −𝑠𝑎𝑙𝑡 𝑝𝑒𝑟𝑚𝑒𝑎𝑏𝑖𝑙𝑖𝑡𝑦 [𝑚·𝑠^(−1) ]
𝜌_𝑤−𝑑𝑒𝑛𝑠𝑖𝑡𝑦 𝑜𝑓 𝑝𝑢𝑟𝑒 𝑤𝑎𝑡𝑒𝑟 [𝑘𝑔·𝑚^(−3)]
Δ𝑃−𝑝𝑟𝑒𝑠𝑢𝑟𝑒 𝑑𝑟𝑜𝑝 [𝑃𝑎]
𝑃_𝑝 -> atmospheric pressure

𝐽𝑤_𝑜𝑢𝑡=𝐴([𝑃_𝑜𝑢𝑡−𝑃_𝑝 ]−〖[𝜋〗_𝑜𝑢𝑡 ]) 𝜌_𝑤
𝐽𝑤_𝑖𝑛=𝐴([𝑃_𝑖𝑛−𝑃_𝑝 ]−[𝜋_𝑖𝑛 ])  𝜌_𝑤
𝐽𝑤_𝑎𝑣𝑔=0.5 (𝐽𝑤_𝑖𝑛+𝐽𝑤_𝑜𝑢𝑡 )
𝑀_(𝑝,𝐻2𝑂)=𝐴𝑟𝑒𝑎 𝐽𝑤_𝑎𝑣𝑔
A = 4.2E-12 # 𝑤𝑎𝑡𝑒𝑟 𝑝𝑒𝑟𝑚𝑒𝑎𝑏𝑖𝑙𝑖𝑡𝑦 coefficient m Pa-1 s-1
B = 3.5E-8 # Salt permeability coefficient m s-1
Maximum hydraulic pressure – 85 bar



Need a separate pump. “RO pump”. Need “energy recovery device” from outlet pressure.

Room temp T good across system. Isothermal. 
No salinity on permeate side. 𝜋_𝑢  calculated. Pi p -> goes to zero.
Water flux as function of inlet and outlet conditions, then average inlet and outlet conditions to get Jwavg, which represts performance across whole space. Get how much water is produced by getting j average times membrane area.

Pressure out of pump -> 65 for 1st stage. For 2nd stage it is dependent on water recovery. Set as unfixed for pressure. 
Pressure in is atmospheric 1. 

Energy recovery device -> reverse of pump.